In [ ]:
"""!wget http://images.cocodataset.org/zips/train2017.zip http://images.cocodataset.org/zips/val2017.zip http://images.cocodataset.org/zips/test2017.zip http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
!mkdir coco_dataset
!mv *.zip coco_dataset/
%cd coco_dataset
!ls
!unzip panoptic_annotations_trainval2017.zip
%cd annotations
!ls
!unzip panoptic_train2017.zip
!unzip panoptic_val2017.zip
%cd ..
!unzip test2017.zip
!unzip train2017.zip
!unzip val2017.zip
!rm *.zip
%cd annotations
!rm *.zip
%cd ..
%cd ..
!mkdir coco_dataset_tf
!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/data/build_coco_data.py \
  --coco_root=coco_dataset \
  --output_dir=coco_dataset_tf"""

'!wget http://images.cocodataset.org/zips/train2017.zip http://images.cocodataset.org/zips/val2017.zip http://images.cocodataset.org/zips/test2017.zip http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip\n!mkdir coco_dataset\n!mv *.zip coco_dataset/\n%cd coco_dataset\n!ls\n!unzip panoptic_annotations_trainval2017.zip\n%cd annotations\n!ls\n!unzip panoptic_train2017.zip\n!unzip panoptic_val2017.zip\n%cd ..\n!unzip test2017.zip\n!unzip train2017.zip\n!unzip val2017.zip\n!rm *.zip\n%cd annotations\n!rm *.zip\n%cd ..\n%cd ..\n!mkdir coco_dataset_tf\n!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/data/build_coco_data.py   --coco_root=coco_dataset   --output_dir=coco_dataset_tf'

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16653872866067802665
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14328594432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14694440924583113987
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
!pip install segments-ai
import os
import re
import gdown
import shutil
from segments import SegmentsClient
from segments import SegmentsDataset
from segments.utils import export_dataset

current_dir=os.getcwd()
os.mkdir('surface_dataset')
os.chdir('surface_dataset')
os.mkdir('annotations')
os.mkdir('train')
os.mkdir('val')
os.chdir('annotations')
os.mkdir('panoptic_train')
os.mkdir('panoptic_val')
os.chdir(current_dir)

url="https://drive.google.com/uc?id=1ANhD30oqlSJDwu0bI3S9AeA2YwOYR891"
gdown.download(url,"data.zip", quiet=True)
shutil.unpack_archive('data.zip',extract_dir='surface_dataset')
os.rename(os.path.join(os.getcwd(),'surface_dataset','test data'),os.path.join(os.getcwd(),'surface_dataset','test'))

api_key = "6b92d0474f7047d8fffa4a8ed0bacec5a41795e1"
segments_client = SegmentsClient(api_key)

"""import matplotlib.pyplot as plt
def store_images(dataset,data_name,type):
    i = 1
    for sample in dataset:
        print("Saving ",data_name," image ", i)
        plt.imsave('surface_dataset/'+data_name+'/' + sample['name'], sample[type])
        i += 1"""

def transfer_photos(pattern,dataset_name,release,destination):
    os.chdir(os.path.join(current_dir,'segments',dataset_name,release))
    l=os.listdir()
    j=1
    for i in l:
        if(re.search(pattern,i)):
            print("Copying file ",j)
            j+=1
            if(pattern=='panoptic'):
                shutil.copy(i,os.path.join(current_dir,'surface_dataset','annotations',destination,i))
            else:
                shutil.copy(i, os.path.join(current_dir, 'surface_dataset', destination, i))
    os.chdir(current_dir)

dataset_identifier1 = "tejash/samsung_sample"
release1 = segments_client.get_release(dataset_identifier1, 'v0.4')
train_dataset = SegmentsDataset(release1, labelset='ground-truth')
#store_images(train_dataset,'train','image')
transfer_photos('^((?!label_ground-truth).)*$','tejash_samsung_sample','v0.4','train')

dataset_identifier2 = "tejash/samsung_validation_set"
release2 = segments_client.get_release(dataset_identifier2, 'v0.2')
val_dataset = SegmentsDataset(release2, labelset='ground-truth')
#store_images(val_dataset,'val','image')
transfer_photos('^((?!label_ground-truth).)*$','tejash_samsung_validation_set','v0.2','val')

#store_images(train_dataset,'annotations/panoptic_train','segmentation_bitmap')
#store_images(val_dataset,'annotations/panoptic_val','segmentation_bitmap')


export_dataset(train_dataset, export_format='coco-panoptic',export_folder=os.path.join(current_dir,'surface_dataset','annotations'))
export_dataset(val_dataset, export_format='coco-panoptic',export_folder=os.path.join(current_dir,'surface_dataset','annotations'))
os.rename(os.path.join(os.getcwd(),'surface_dataset','annotations','export_coco-panoptic_tejash_samsung_sample_v0.4.json'),os.path.join(os.getcwd(),'surface_dataset','annotations','panoptic_train.json'))
os.rename(os.path.join(os.getcwd(),'surface_dataset','annotations','export_coco-panoptic_tejash_samsung_validation_set_v0.2.json'),os.path.join(os.getcwd(),'surface_dataset','annotations','panoptic_val.json'))

transfer_photos('panoptic','tejash_samsung_sample','v0.4','panoptic_train')
transfer_photos('panoptic','tejash_samsung_validation_set','v0.2','panoptic_val')
print(len(os.listdir(os.path.join(os.getcwd(),'surface_dataset','annotations','panoptic_train'))))
print(len(os.listdir(os.path.join(os.getcwd(),'surface_dataset','annotations','panoptic_val'))))
print(len(os.listdir(os.path.join(os.getcwd(),'surface_dataset','train'))))
print(len(os.listdir(os.path.join(os.getcwd(),'surface_dataset','test'))))
print(len(os.listdir(os.path.join(os.getcwd(),'surface_dataset','val'))))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 7.3 MB/s eta 0:00:00
  Created wheel for segments-ai: filename=segments_ai-1.0.25-py3-none-any.whl size=37974 sha256=3bc41cb0aa9d44dce7da89ea5a0ae4961896535697cfe3abb4d18a989efbb43d
  Stored in directory: /root/.cache/pip/wheels/12/0f/c2/cb99bb9cf34f80a4b15e5debd8e69983168efc20c06bfd8076
Successfully built segments-ai
Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 118/118 [00:55<00:00,  2.11it/s]


Initialized dataset with 118 images.
Copying file  1
Copying file  2
Copying file  3
Copying file  4
Copying file  5
Copying file  6
Copying file  7
Copying file  8
Copying file  9
Copying file  10
Copying file  11
Copying file  12
Copying file  13
Copying file  14
Copying file  15
Copying file  16
Copying file  17
Copying file  18
Copying file  19
Copying file  20
Copying file  21
Copying file  22
Copying file  23
Copying file  24
Copying file  25
Copying file  26
Copying file  27
Copying file  28
Copying file  29
Copying file  30
Copying file  31
Copying file  32
Copying file  33
Copying file  34
Copying file  35
Copying file  36
Copying file  37
Copying file  38
Copying file  39
Copying file  40
Copying file  41
Copying file  42
Copying file  43
Copying file  44
Copying file  45
Copying file  46
Copying file  47
Copying file  48
Copying file  49
Copying file  50
Copying file  51
Copying file  52
Copying file  53
Copying file  54
Copying file  55
Copying file  56
Copying file  57
Cop

100%|██████████| 5/5 [00:01<00:00,  3.46it/s]


Initialized dataset with 5 images.
Copying file  1
Copying file  2
Copying file  3
Copying file  4
Copying file  5
Exporting dataset. This may take a while...


100%|██████████| 118/118 [01:12<00:00,  1.64it/s]


Exported to /content/surface_dataset/annotations/export_coco-panoptic_tejash_samsung_sample_v0.4.json. Images and labels in segments/tejash_samsung_sample/v0.4
Exporting dataset. This may take a while...


100%|██████████| 5/5 [00:00<00:00, 25.20it/s]

Exported to /content/surface_dataset/annotations/export_coco-panoptic_tejash_samsung_validation_set_v0.2.json. Images and labels in segments/tejash_samsung_validation_set/v0.2
Copying file  1
Copying file  2
Copying file  3
Copying file  4
Copying file  5
Copying file  6
Copying file  7
Copying file  8
Copying file  9
Copying file  10
Copying file  11
Copying file  12
Copying file  13
Copying file  14
Copying file  15
Copying file  16
Copying file  17
Copying file  18
Copying file  19
Copying file  20
Copying file  21
Copying file  22
Copying file  23
Copying file  24
Copying file  25
Copying file  26
Copying file  27
Copying file  28
Copying file  29
Copying file  30
Copying file  31
Copying file  32
Copying file  33
Copying file  34
Copying file  35
Copying file  36
Copying file  37
Copying file  38
Copying file  39
Copying file  40
Copying file  41
Copying file  42
Copying file  43
Copying file  44
Copying file  45
Copying file  46
Copying file  47
Copying file  48
Copying file  49


In [ ]:
from PIL import Image
l=os.listdir('surface_dataset/train')

In [ ]:
for i in l:
  name,extension=tuple(os.path.splitext('surface_dataset/train/'+i))
  x=Image.open('surface_dataset/train/'+i)
  if(extension=='.jpeg'):
    x.save(name+'.jpg')
    os.remove('surface_dataset/train/'+i)
  elif(extension=='.png'):
    x=x.convert('RGB')
    x.save(name+'.jpg')
    os.remove('surface_dataset/train/'+i)

In [ ]:
new_l=os.listdir('surface_dataset/train')
print(len(new_l))
for i in new_l:
  extension=os.path.splitext('surface_dataset/train/'+i)[1]
  if(extension!='.jpg'):
    print('error')

118


In [ ]:
import json
with open('surface_dataset/annotations/panoptic_train.json','r') as f:
  pt=json.load(f)
new_pt={}
for i in list(pt.keys()):
  if(i!='images'):
    new_pt[i]=pt[i]
image_dict_list=pt['images']
for i in range(len(image_dict_list)):
  name,ext=os.path.splitext(image_dict_list[i]['file_name'])
  if(ext!='.jpg'):
    ext='.jpg'
  image_dict_list[i]['file_name']=name+ext
new_pt['images']=image_dict_list

In [ ]:
os.remove('surface_dataset/annotations/panoptic_train.json')
with open('surface_dataset/annotations/panoptic_train.json','w') as f:
  json.dump(new_pt,f)

In [ ]:
with open('surface_dataset/annotations/panoptic_train.json','r') as f:
  x=json.load(f)
x

{'info': {'description': 'samsung_sample', 'version': 'v0.4'},
 'categories': [{'id': 1,
   'name': 'horizontal_surface',
   'color': [0, 113, 188],
   'attributes': None,
   'isthing': 0},
  {'id': 2,
   'name': 'vertical_surface',
   'color': [216, 82, 24],
   'attributes': None,
   'isthing': 0}],
 'annotations': [{'segments_info': [{'id': 12349696,
     'category_id': 1,
     'bbox': [0, 3003, 2592, 1605],
     'area': 3142136,
     'iscrowd': 0},
    {'id': 1594072,
     'category_id': 2,
     'bbox': [0, 0, 2592, 4278],
     'area': 8801800,
     'iscrowd': 0}],
   'file_name': 'IMG20230603100909_label_ground-truth_coco-panoptic.png',
   'image_id': 1},
  {'segments_info': [{'id': 12349696,
     'category_id': 1,
     'bbox': [61, 0, 2531, 4608],
     'area': 4550590,
     'iscrowd': 0},
    {'id': 1594072,
     'category_id': 2,
     'bbox': [0, 0, 2592, 4608],
     'area': 7393346,
     'iscrowd': 0}],
   'file_name': 'IMG20230603100303_label_ground-truth_coco-panoptic.png',
  

In [ ]:
!git clone https://github.com/TXH2020/deeplab2.git

Cloning into 'deeplab2'...
remote: Enumerating objects: 2331, done.
remote: Counting objects: 100% (1026/1026), done.
remote: Compressing objects: 100% (259/259), done.
remote: Total 2331 (delta 857), reused 767 (delta 767), pack-reused 1305
Receiving objects: 100% (2331/2331), 8.45 MiB | 5.23 MiB/s, done.
Resolving deltas: 100% (1644/1644), done.


In [ ]:
!git clone https://github.com/txh2020/models.git

Cloning into 'models'...
remote: Enumerating objects: 78743, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 78743 (delta 56), reused 73 (delta 50), pack-reused 78658
Receiving objects: 100% (78743/78743), 597.60 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (56391/56391), done.


In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [ ]:
%cd cocoapi/PythonAPI/
!make
%cd ..
%cd ..

/content/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-cpython-310
creating build/temp.linux-x86_64-cpython-310/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/numpy/core/include -I../common -I/usr/include/python3.10 -c ../common/maskApi.c -o build/temp.linux-x86_64-cpython-310/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=

In [ ]:
!protoc deeplab2/*.proto --python_out=.

In [ ]:
!pip install immutabledict

In [ ]:
#!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/model/deeplab_test.py

In [ ]:
#!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/trainer/evaluator_test.py

In [ ]:
!mkdir surface_dataset_tf

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/data/build_coco_data.py \
  --coco_root=surface_dataset \
  --output_dir=surface_dataset_tf

2023-07-06 15:00:08.710123: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 15:00:09.609930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-06 15:00:11.981080: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-06 15:00:12.603339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/trainer/train.py --config_file=deeplab2/configs/cityscapes/panoptic_deeplab/mobilenet_v3_small_os16.textproto --mode=train --model_dir=training_output --num_gpus=1

2023-07-06 15:20:34.824983: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 15:20:37.121515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-06 15:20:41.733840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-06 15:20:42.265284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
!tar -cvf train_heavy.tar training_output

training_output/
training_output/Deeplab/
training_output/Deeplab/eval/
training_output/Deeplab/eval/events.out.tfevents.1688657285.6e3f937dece3.14450.0.v2
training_output/Deeplab/ckpt-100.data-00000-of-00001
training_output/Deeplab/ckpt-0.data-00000-of-00001
training_output/Deeplab/train/
training_output/Deeplab/train/events.out.tfevents.1688656860.6e3f937dece3.11827.0.v2
training_output/Deeplab/train/events.out.tfevents.1688656251.6e3f937dece3.7019.0.v2
training_output/Deeplab/train/events.out.tfevents.1688655744.6e3f937dece3.3404.0.v2
training_output/Deeplab/train/events.out.tfevents.1688656676.6e3f937dece3.10296.0.v2
training_output/Deeplab/train/events.out.tfevents.1688656443.6e3f937dece3.8557.0.v2
training_output/Deeplab/train/events.out.tfevents.1688656046.6e3f937dece3.5410.0.v2
training_output/Deeplab/checkpoint
training_output/Deeplab/ckpt-0.index
training_output/Deeplab/vis/
training_output/Deeplab/vis/000000_semantic_error.png
training_output/Deeplab/vis/000000_semantic_labe

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/trainer/train.py --config_file=deeplab2/configs/cityscapes/panoptic_deeplab/mobilenet_v3_small_os16.textproto --mode=eval --model_dir=training_output --num_gpus=1

2023-07-06 15:27:55.629842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 15:27:56.630152: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-06 15:27:58.631642: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-06 15:27:58.674041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd` && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI && python deeplab2/export_model.py --experiment_option_path=deeplab2/configs/cityscapes/panoptic_deeplab/mobilenet_v3_small_os16.textproto --checkpoint_path=training_output/Deeplab/ckpt-100 --output_path=mymodel

2023-07-06 15:33:17.776360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 15:33:18.745361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-06 15:33:20.761996: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-06 15:33:20.797169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('mymodel')

In [ ]:
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
